In [1]:
import requests
import pandas as pd
from getpass import getpass
import os
import time
from pymongo import MongoClient
from bson.decimal128 import Decimal128
from collections import defaultdict
from dotenv import load_dotenv
load_dotenv()
import json
import geojson
from cartoframes.viz import Map, Layer, popup_element
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from folium import Icon
from folium import plugins

## PROCESS AND SOLUTIONS ##

**IMPORT SOURCE DOCUMENTS**

In [2]:
import src.exploration as exp
import src.forsquare as fsq
import src.visualization as vis

**EXPLORATION FUNCTIONS**

In [3]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")

    FINDING COMPANIES WITH 2 OF THE REQUIREMENTS OF EMPLOYEES:
    
        - design companies in the city
        - successful startups

In [4]:
design_companies = exp.find_design_companies(c)
successful_startups = exp.find_successful_startups(c)
city_counts = exp.oc_city_counts(successful_startups, "offices", "city")
city_counts_design= exp.oc_city_counts(design_companies, "offices", "city")

In [5]:
city_counts.head()

,Count
San Francisco,190
New York,127
London,78
Seattle,43
Palo Alto,41


In [6]:
city_counts_design

,Count
San Francisco,1
Collingwood,1
Berlin,1
London,1
Brooklyn,1
Ellensburg,1


FINDINGS

From what I found out, the top 3 cities to place the company are, San Francisco, New York, and London. 

In those cities there are the greatest number of successful startups and other Design companies to satisfy the Designers requirements.

But as the top 2 are in the States, instead of London I think it's better to find another city in the states to filter from. In that case the next option it's going to be Seattle. 

**GEOJSON AND 4SQR**

In [7]:
client = MongoClient("localhost:27017")
db = client["cities_project"]

In [8]:
token = os.getenv("token")

    GETTING LOCATION FOR EACH CITY

In [ ]:
san_francisco_loc = fsq.get_coordinates("501 Twin Peaks Blvd, San Francisco, CA 94114, Estados Unidos")
new_york_loc = fsq.get_coordinates("Central Park N, New York, New York")
seattle_loc = fsq.get_coordinates("1027 S Washington St, Seattle, WA 98104, EE. UU.")

In [9]:
san_francisco_loc = [37.75404,-122.44721]
new_york_loc = [40.79724,-73.94968]
seattle_loc = [47.60065,-122.31819]

    FINDING REQUIREMENTS FOR EACH CITY AND UPLOADING THEM TO MONGO

    - SAN FRANCISCO

In [ ]:
starbucks_SF = fsq.foursquare_places ("Starbucks",san_francisco_loc, "SFstarbucks")
basketball_stadiums_SF = fsq.foursquare_places ("Basketball stadium",san_francisco_loc, "SFbaskStadium")
schools_SF = fsq.foursquare_places ("schools", san_francisco_loc, "SFschools")
airport_SF = fsq.foursquare_places ("airport",san_francisco_loc,"SFairport")
train_stations_SF = fsq.foursquare_places ("train station",san_francisco_loc, "SFtrainStations")
pet_grooming_SF = fsq.foursquare_places ("pet grooming", san_francisco_loc,"SFpet_grooming")
dog_parks_SF = fsq.foursquare_places ("dog park", san_francisco_loc,"SFdog_parks")
bars_clubs_SF = fsq.foursquare_places ("night club",san_francisco_loc,"SFbars_clubs")
vegan_vegetarian_SF = fsq.foursquare_places ("vegan restaurant", san_francisco_loc,"SFvegan_vegetarian")

    - NEW YORK CITY

In [ ]:
starbucks_NYC = fsq.foursquare_places ("Starbucks",new_york_loc,"NYCstarbucks")
basketball_stadiums_NYC = fsq.foursquare_places ("Basketball stadium",new_york_loc,"NYCbaskStadium")
schools_NYC = fsq.foursquare_places ("schools",new_york_loc,"NYCschools")
airport_NYC = fsq.foursquare_places ("airport",new_york_loc,"NYCairport")
train_stations_NYC = fsq.foursquare_places ("train station",new_york_loc,"NYCtrain_stations")
pet_grooming_NYC = fsq.foursquare_places ("pet grooming",new_york_loc,"NYCpet_grooming")
dog_parks_NYC = fsq.foursquare_places ("dog park",new_york_loc,"NYCdog_parks")
bars_clubs_NYC = fsq.foursquare_places ("night club",new_york_loc,"NYCbars_clubs")
vegan_vegetarian_NYC = fsq.foursquare_places ("vegan restaurant",new_york_loc,"NYCvegan_vegetarian")

     - SEATTLE

In [ ]:
starbucks_ST = fsq.foursquare_places ("Starbucks",seattle_loc,"STstarbucks")
basketball_stadiums_ST = fsq.foursquare_places ("Basketball stadium",seattle_loc,"STbaskStadium")
schools_ST = fsq.foursquare_places ("schools",seattle_loc,"STschools")
airport_ST = fsq.foursquare_places ("airport",seattle_loc,"STairport")
train_stations_ST = fsq.foursquare_places ("train station",seattle_loc,"STtrain_stations")
pet_grooming_ST = fsq.foursquare_places ("pet grooming",seattle_loc, "STpet_grooming")
dog_parks_ST = fsq.foursquare_places ("dog park",seattle_loc,"STdog_parks")
bars_clubs_ST = fsq.foursquare_places ("night club",seattle_loc,"STbars_clubs")
vegan_vegetarian_ST = fsq.foursquare_places ("vegan restaurant",seattle_loc,"STvegan_vegetarian")

Once we've got all the collections uploaded to mongo we can start looking at the data and plotting.

**VISUALIZATION**

    IMPORTING MONGO COLLECTIONS OF REQUIREMENTS

    - SAN FRANCISCO

In [10]:
SFstarbucks = vis.mongodb_to_df("SFstarbucks")
SFbaskStadium = vis.mongodb_to_df("SFbaskStadium")
SFschools = vis.mongodb_to_df("SFschools")
SFairport = vis.mongodb_to_df("SFairport")
SFtrainStations = vis.mongodb_to_df("SFtrainStations")
SFpet_grooming = vis.mongodb_to_df("SFpet_grooming")
SFdog_parks = vis.mongodb_to_df("SFdog_parks")
SFbars_clubs = vis.mongodb_to_df("SFbars_clubs")
SFvegan_vegetarian = vis.mongodb_to_df("SFvegan_vegetarian")

    - NEW YORK CITY

In [11]:
NYCstarbucks = vis.mongodb_to_df("NYCstarbucks")
NYCbaskStadium = vis.mongodb_to_df("NYCbaskStadium")
NYCschools = vis.mongodb_to_df("NYCschools")
NYCairport = vis.mongodb_to_df("NYCairport")
NYCtrainStations = vis.mongodb_to_df("NYCtrain_stations")
NYCpet_grooming = vis.mongodb_to_df("NYCpet_grooming")
NYCdog_parks = vis.mongodb_to_df("NYCdog_parks")
NYCbars_clubs = vis.mongodb_to_df("NYCbars_clubs")
NYCvegan_vegetarian = vis.mongodb_to_df("NYCvegan_vegetarian")

    - SEATTLE

In [12]:
STstarbucks = vis.mongodb_to_df("STstarbucks")
STbaskStadium = vis.mongodb_to_df("STbaskStadium")
STschools = vis.mongodb_to_df("STschools")
STairport = vis.mongodb_to_df("STairport")
STtrainStations = vis.mongodb_to_df("STtrain_stations")
STpet_grooming = vis.mongodb_to_df("STpet_grooming")
STdog_parks = vis.mongodb_to_df("STdog_parks")
STbars_clubs = vis.mongodb_to_df("STbars_clubs")
STvegan_vegetarian = vis.mongodb_to_df("STvegan_vegetarian")

    IMPORT AND READ GEOJSONS OF THE CITIES  

In [13]:
SF_gj = vis.read_geojson("data/SanFrancisco.Neighborhoods.json")
NYC_gj = vis.read_geojson("data/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson")
ST_gj = vis.read_geojson("data/Neighborhood_Map_Atlas_Neighborhoods.geojson")

- HEAT MAPS TO COMPARE DENSITY OF REQUIREMENTS IN EACH CITY

    SAN FRANCISCO

In [14]:
SFheatmap = vis.heat_map("SFheatmap",san_francisco_loc, SF_gj, SFstarbucks,SFbaskStadium, SFschools, SFairport, SFtrainStations, SFpet_grooming, SFdog_parks, SFbars_clubs,SFvegan_vegetarian)
SFheatmap

    NEW YORK CITY

In [15]:
NYCheatmap = vis.heat_map("NYCheatmap",new_york_loc, NYC_gj, NYCstarbucks,NYCbaskStadium, NYCschools, NYCairport, NYCtrainStations, NYCpet_grooming, NYCdog_parks, NYCbars_clubs,NYCvegan_vegetarian)
NYCheatmap

    SEATTLE

In [16]:
STheatmap = vis.heat_map("STheatmap",seattle_loc, ST_gj, STstarbucks,STbaskStadium, STschools, STairport, STtrainStations, STpet_grooming, STdog_parks, STbars_clubs,STvegan_vegetarian)
STheatmap

From what we can see in these heat maps, we can already discard Seattle, as most of the city does not have our requirements and also we've seen previously that there is no Design company in the city.

Although New York has more heated points in different places of the city, there's also good density of requirements in San Francisco.

To decide between New York and San Francisco, we can compare the airports and train stations distances.

- NEW YORK VS SAN FRANCISCO

    NEW YORK

In [17]:
NYtransport_map = vis.travel("NYtransport_map", new_york_loc, NYC_gj, NYCairport, NYCtrainStations)
NYtransport_map

    SAN FRANCISCO

In [18]:
SFtransport_map = vis.travel("SFtransport_map", san_francisco_loc, SF_gj, SFairport, SFtrainStations)
SFtransport_map

For what we can see in those 2 maps. San Francisco doesn't have a close airport, it doesn't even appear in the map, as it only shows what is inside the borders of the geo json.
And the way to get public transport to get to the aiport, is not optimal. There are not as many possibilities.

Insted in New York, there are at least 3 closer airports, and a huge multitude of ways to get there, from a lot of places in the city. 

Most of them Manhattan.

This is why, the city I'm going to choose to put my company is New York, specifically Manhattan. 

The next step is to find the best neighborhood.

- NEW YORK MANHATTAN NEIGHBORHOODS

In [19]:
NYselectmap = vis.NY_complete("NYselectmap", new_york_loc, NYC_gj, NYCstarbucks,NYCbaskStadium, NYCschools, NYCairport, NYCtrainStations, NYCpet_grooming, NYCdog_parks, NYCbars_clubs,NYCvegan_vegetarian)
NYselectmap

To place only the markers inside of the borders of NYC I've used a Geojson and said only to display the markers within these borders.

For the CEO, looking only at vegan restaurants, and starbucks almost all of Manhattan works. But maybe better Upper West Side, Chelsea and Union Square.

For clubs it's either Chelsea or the Lower East Side.

For transportation it's either Chelsea or Midtown.

For Schools it's the Upper West Side and, West Village. Chelsea is in between of those 2 neghborhoods, so not that far.

For Stadiums it's also Chelsea.

For dog parks, the Upper West, Chelsea and Union Square are a good Option.

And for pet grooming it would be Upper West, Upper East and Union Square.

    So as for the data we've got it looks like the best option is Chelsea. Also Chelsea is close to Brooklyn, and there is a Design company there. 

Now that we've got our final decision, let's show in a map how all the requirements would show on the chelsea neighbourhood.

- CHELSEA

In [20]:
chelsea_map =  vis.chelsea_select(NYC_gj,"Chelsea_map",NYCstarbucks,NYCbaskStadium, NYCschools, NYCairport, NYCtrainStations, NYCpet_grooming, NYCdog_parks, NYCbars_clubs,NYCvegan_vegetarian)
chelsea_map

/Users/emma/Desktop/Emma/ironhack_data/projects/Company_Loc/src/visualization.py:382: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map = folium.Map(location=[chelsea_neighborhood.geometry.centroid.y.mean(), chelsea_neighborhood.geometry.centroid.x.mean()], zoom_start=14)


The 28th with 7th Ave looks like a great location. Its 2 streets for coffe, 3 for train, 3 for vegan restaurants and a lot more that doesn't appear on the map. It's next to Madison Square Park, Herald, Square. There are a lot of Metro Stations, restaurants, rooftops, and a hole lot more starbucks. It's one of the bests locations in the city for sure. 

    Now that we've chosen a possible location let's do a more thorough search on the area.

    - SPECIFIC FORSQUARE SEARCH

In [ ]:
chealse_loc = fsq.get_coordinates("320 7th Ave, New York, NY 10001, EE. UU")

In [ ]:
starbucks_Chelsea = fsq.foursquare_places ("Starbucks",chealse_loc,"Chelsea_starbucks")
stadiums_Chelsea = fsq.foursquare_places ("Baketball stadium",chealse_loc,"Chelsea_stadiums")
schools_Chelsea = fsq.foursquare_places ("school",chealse_loc,"Chelsea_school")
airport_Chelsea = fsq.foursquare_places ("airport",chealse_loc,"Chelsea_airport")
train_stations_Chelsea = fsq.foursquare_places ("train station",chealse_loc,"Chelsea_train_stations")
dog_park_Chelsea = fsq.foursquare_places ("dog park",chealse_loc,"Chelsea_dog_park")
bars_clubs_Chelsea = fsq.foursquare_places ("night club",chealse_loc,"Chelsea_bars_clubs")
vegan_vegetarian_Chelsea = fsq.foursquare_places ("vegan restaurant",chealse_loc,"Chelsea_vegan_vegetarian")

    - IMPORT COLLECTIONS FROM MONGO

In [21]:
CSstarbucks = vis.mongodb_to_df("Chelsea_starbucks")
CSbaskStadium = vis.mongodb_to_df("Chelsea_stadiums")
CSschools = vis.mongodb_to_df("Chelsea_school")
CSschools = vis.mongodb_to_df("Chelsea_airport")
CStrainStations = vis.mongodb_to_df("Chelsea_train_stations")
CSCSdog_parks = vis.mongodb_to_df("Chelsea_dog_park")
CSbars_clubs = vis.mongodb_to_df("Chelsea_bars_clubs")
CSvegan_vegetarian = vis.mongodb_to_df("Chelsea_vegan_vegetarian")

In [22]:
detail_chelsea_map =  vis.chelsea_select(NYC_gj,"Detail_Chelsea_map",CSstarbucks,CSbaskStadium, CSschools, NYCairport, NYCtrainStations, NYCpet_grooming, CSCSdog_parks, CSbars_clubs,CSvegan_vegetarian)
detail_chelsea_map

/Users/emma/Desktop/Emma/ironhack_data/projects/Company_Loc/src/visualization.py:382: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map = folium.Map(location=[chelsea_neighborhood.geometry.centroid.y.mean(), chelsea_neighborhood.geometry.centroid.x.mean()], zoom_start=14)


Doing a more thorough investigation, we can see that there's a bunch of schools right next to the chosen location, and dog parks. Also Starbucks only a minute away. More variety of vegan restaurants, and a hole lot more of night clubs. That on top of the already found stadiums, train stations and ways to get to the airport. All in all, I think we've got our perfect location. 

All that within the borders of the Chelsea Neighborhood geo json.